In [1]:
import pandas as pd, numpy as np
import librosa, os

## Create initial data frame
**store filenames, filepaths, emotion code as label**

In [27]:
#create full filepaths
d = "..\\SER_IGP_Project\\RAVDESS\\audio_speech_actors_01-24"
filepaths = []
for path, subdirs, files in os.walk(d):
    for name in files:
        filepaths.append(os.path.join(path, name))
        
#filenames and labels
df=pd.DataFrame()
df['path'] = [x for x in filepaths]
df['file'] = [x[-24:] for x in filepaths] #populate filenames
df['label'] = [x.split('-')[2] for x in df['file']] #read emotion code and set as label
df=df.loc[(df["label"]=='01')|(df["label"]=='03')|(df["label"]=='04')].reset_index(drop=True) #filter neutral, happy, sad

## feature data
**store all extracted feature data    
limited to 50 observations to reduce processing time    
output to csv**

In [35]:
#set empty lists to store data
stft,mel,cent,rms,mfcc,bw,contrast,flat,rolloff,zcr=[],[],[],[],[],[],[],[],[],[]

#iterate files to extract features
for x in df['path'][:50]: #note: limited to 10 to reduce processing time and out put size
    y, sr = librosa.load(x) #set file and  sample rate
    stft.append(librosa.stft(y))
    mfcc.append(librosa.feature.mfcc(y=y,sr=sr))
    rms.append(librosa.feature.rms(y=y))
    cent.append(librosa.feature.spectral_centroid(y=y, sr=sr))
    bw.append(librosa.feature.spectral_bandwidth(y=y,sr=sr))
    contrast.append(librosa.feature.spectral_contrast(y=y,sr=sr))
    flat.append(librosa.feature.spectral_flatness(y=y))
    rolloff.append(librosa.feature.spectral_rolloff(y=y,sr=sr))
    zcr.append(librosa.feature.zero_crossing_rate(y=y))
    mel.append(librosa.feature.melspectrogram(y=y,sr=sr))
#store all extracted data as df
df1 = pd.DataFrame({'stft':stft,
                    'melspec':mel,
                    'spec_centroid':cent,
                    'root_mean_square_energy':rms,
                    'mfcc':mfcc,
                    'spectral_bandwidth':bw,
                    'spectral_contrast':contrast,
                    'spectral_flatness':flat,
                    'roll_off':rolloff,
                    'zero_crossing':zcr,})

#concatenate label + all vectors to one df
vectordf = pd.concat([df['label'][:50], df1], axis=1)

#output to csv
vectordf.to_csv('all_features.csv',index=False)